In [44]:
import numpy as np
from sklearn.base import BaseEstimator
from sklearn.metrics import euclidean_distances
from sklearn.utils import check_random_state, check_array, check_symmetric
#from sklearn.externals.joblib import Parallel, delayed, effective_n_jobs
from sklearn.isotonic import IsotonicRegression

def NMDSfunc(dissimilarities, n_dimensions=2, max_iter=300, verbose=False, eps=1e-3, random_state=None):
    '''
    dissimilarities: matrix
    eps: used for convergence
    '''
    dissimilarities = check_symmetric(dissimilarities, raise_exception=True)
    n_samples = dissimilarities.shape[0]
    random_state = check_random_state(random_state)

    sim_flat = ((1 - np.tri(n_samples)) * dissimilarities).ravel()
    sim_flat_w = sim_flat[sim_flat != 0]
    #Random initial configuration
    X = random_state.rand(n_samples * n_dimensions)
    X = X.reshape((n_samples, n_dimensions))

    old_stress = None
    ir = IsotonicRegression()
    for it in range(max_iter):
        # Compute distance and monotonic regression
        dis = euclidean_distances(X)
        dis_flat = dis.ravel()
        dis_flat_w = dis_flat[sim_flat != 0]

        # Finf disparities using monotonic regression
        disparities_flat = ir.fit_transform(sim_flat_w, dis_flat_w)
        disparities = dis_flat.copy()
        disparities[sim_flat != 0] = disparities_flat
        disparities = disparities.reshape((n_samples, n_samples))
        disparities *= np.sqrt((n_samples * (n_samples - 1) / 2) / (disparities ** 2).sum())
        # Compute stress
        stress = ((dis.ravel() - disparities.ravel()) ** 2).sum() / 2

        # Update X using the Guttman transform
        dis[dis == 0] = 1e-5
        ratio = disparities / dis
        B = - ratio
        B[np.arange(len(B)), np.arange(len(B))] += ratio.sum(axis=1)
        X = 1. / n_samples * np.dot(B, X)

        dis = np.sqrt((X ** 2).sum(axis=1)).sum()
        if verbose:
            print('iteration: %d, stress %s' % (it, stress))
        if old_stress is not None:
            if(old_stress - stress / dis) < eps:
                if verbose:
                    print('convergence: breaking at iteration %d with stress %s' % (it,stress))
                break
        old_stress = stress / dis

    return X, stress, it + 1


def NMDS(dissimilarities, n_dimensions=2, n_init=4,
            max_iter=300, verbose=False, eps=1e-3, random_state=None):
    """
    """
    dissimilarities = check_array(dissimilarities)
    random_state = check_random_state(random_state)

    best_pos, best_stress = None, None
    for it in range(n_init):
        pos, stress, n_iter_ = NMDSfunc(dissimilarities, n_dimensions=n_dimensions, max_iter=max_iter, verbose=verbose,
                eps=eps, random_state=random_state)
        print(pos)
        print(stress)
        if best_stress is None or stress < best_stress:
            best_stress = stress
            best_pos = pos.copy()
            best_iter = n_iter_
            
    return best_pos, best_stress, best_iter


class MDS():
    def __init__(self, n_dimensions=2,  n_init=4,
                 max_iter=300, verbose=False, eps=1e-3,
                 random_state=None):
        self.n_dimensions = n_dimensions
        self.n_init = n_init
        self.max_iter = max_iter
        self.eps = eps
        self.verbose = verbose
        self.random_state = random_state

    def fit(self, X, y=None):
      
        #self.fit_transform(X)
        X = check_array(X)
        self.dissimilarity_matrix_ = X
        
        self.embedding, self.stress, self.n_iter = NMDS(self.dissimilarity_matrix_,
            n_dimensions=self.n_dimensions, n_init=self.n_init,
            max_iter=self.max_iter, verbose=self.verbose,
            eps=self.eps, random_state=self.random_state)
        
        return self

In [51]:
sim = np.array([[0, 5, 3, 4],
                    [5, 0, 2, 2],
                    [3, 2, 0, 1],
                    [4, 2, 1, 0]])
mds_clf = MDS()
a = mds_clf.fit(sim)

[[-0.09929721 -0.67767375]
 [ 0.23075424  0.18620914]
 [-0.17561697  0.09403646]
 [ 0.0409847   0.38055228]]
0.0109563293289
[[-0.5541385   0.28148819]
 [ 0.35789969 -0.43610676]
 [ 0.13669892 -0.00865695]
 [ 0.05886924  0.16040147]]
0.0125482922792
[[-0.24872573 -0.13719767]
 [ 0.08729399 -0.35247961]
 [-0.32774578  0.32313976]
 [ 0.49749066  0.19927699]]
0.0937225465117
[[ 0.37078889 -0.42915919]
 [-0.17362818  0.46701517]
 [ 0.15403576  0.08859029]
 [-0.36925985 -0.13103964]]
0.00540617827044


In [52]:
a.stress

0.0054061782704412338

In [53]:
a.embedding

array([[ 0.37078889, -0.42915919],
       [-0.17362818,  0.46701517],
       [ 0.15403576,  0.08859029],
       [-0.36925985, -0.13103964]])